In [17]:
import copy
import random
import pandas as pd
import openai
import os
from PyPDF2 import PdfReader
import json

CHATGPT_MODEL = "gpt-4o"
client = None
MAX_LENGTH = 4096

NUM_QUESTIONS = 50 

def set_openai_api_key(api_key):
    global client
    client = openai
    client.api_key = api_key


set_openai_api_key(input("Please enter your API key: "))

In [46]:
def generate_question(type, tips):
    messages = [
            {"role": "system", "content": f"Your job is to create {type} that users would normally ask."},
            {"role": "user",
             "content": f"Generate {NUM_QUESTIONS} questions in which you will ask the chatbot to create an expense for you from invoice {tips}:\n[\n[{{\"role\": \"user\", \"content\": \"question 1\"}}],\n[{{\"role\": \"user\", \"content\": \"question 2\"}}],\n... \n] don't write ```json"}
        ]
    return messages

In [47]:
def generate_question_about_invoice_status(type, tips):
    messages = [
            {"role": "system", "content": f"Your job is to create {type} that users would normally ask."},
            {"role": "user",
             "content": f"Generate {NUM_QUESTIONS} questions in which you will ask the chatbot what is status of [Fake number] expense, made up some fake number that could be used as expense id  {tips}:\n[\n[{{\"role\": \"user\", \"content\": \"question 1\"}}],\n[{{\"role\": \"user\", \"content\": \"question 2\"}}],\n... \n] don't write ```json"}
        ]
    return messages

In [48]:
def generate_questions_from_text(messages):
    response = client.chat.completions.create(
        model=CHATGPT_MODEL,
        messages=messages,
        max_tokens=MAX_LENGTH,
    )

    try:
        print(response.choices[0].message.content)
        synthetic_data = json.loads(response.choices[0].message.content)
        print(synthetic_data)
        return synthetic_data
    except json.JSONDecodeError:
        try:
            print("Response not in JSON format. Asking ChatGPT to correct it.")
            correction_messages = [
                {"role": "system", "content": "Your job is to correct the format of the questions."},
                {"role": "user",
                 "content": f"The previous response was not in the correct format. Ensure the response is in the following JSON format:\n[\n[{{\"role\": \"user\", \"content\": \"question 1\"}}],\n[{{\"role\": \"user\", \"content\": \"question 2\"}}],\n... \n]\nHere are the questions to reformat:\n{response.choices[0].message['content']}  don't write ```json"}
            ]

            response = client.chat.completions.create(
                model=CHATGPT_MODEL,
                messages=correction_messages,
                max_tokens=MAX_LENGTH,
            )

            synthetic_data = json.loads(response.choices[0].message.content)

            return synthetic_data
        except Exception as e:
            return []

In [49]:
data_for_messages =  [
    {
        "type_of_question": " order",
        "tips": "(don't use term invoice in text)"
    }
]
#["short order impolite"]#, "Formal", "informal", "order", "short"]


In [50]:
for message_data  in data_for_messages:
    print(generate_questions_from_text(generate_question(message_data["type_of_question"], message_data["tips"])))

[
[{"role": "user", "content": "Could you create an expense for the purchase made on date X?"}],

[{"role": "user", "content": "Please add an expense for the office supplies bought last week."}],

[{"role": "user", "content": "Generate an expense entry for the recent payment to the vendor."}],

[{"role": "user", "content": "Can you log an expense for the services we acquired on date Y?"}],

[{"role": "user", "content": "I need an expense created for the software subscription purchased yesterday."}],

[{"role": "user", "content": "Create an expense record for our recent travel expenditure."}],

[{"role": "user", "content": "Add an expense for the consultancy fees payable last month."}],

[{"role": "user", "content": "Please register an expense for the marketing services we outsourced."}],

[{"role": "user", "content": "Could you make an expense entry for the new office furniture we bought?"}],

[{"role": "user", "content": "Log an expense for the dinner meeting with the client."}],

[{"

In [53]:
data_for_messages_expense_status =  [
    {
        "type_of_question": " order",
        "tips": "(don't use term id in text)"
    }
]

In [54]:
for message_data  in data_for_messages_expense_status:
    print(generate_questions_from_text(generate_question_about_invoice_status(message_data["type_of_question"], message_data["tips"])))

[
  [{"role": "user", "content": "Can you check the status of expense number 47821?"}],
  [{"role": "user", "content": "What's happening with expense 98347?"}],
  [{"role": "user", "content": "How is expense 56219 progressing?"}],
  [{"role": "user", "content": "Is expense 85120 approved yet?"}],
  [{"role": "user", "content": "What is the current state of expense 42578?"}],
  [{"role": "user", "content": "Could you provide an update on expense 76301?"}],
  [{"role": "user", "content": "What's the status of expense 35286 today?"}],
  [{"role": "user", "content": "Can you look up the status for expense 19475?"}],
  [{"role": "user", "content": "Has there been any progress on expense 92257?"}],
  [{"role": "user", "content": "I'd like to know what's going on with expense 47892."}],
  [{"role": "user", "content": "What stage is expense 29471 at right now?"}],
  [{"role": "user", "content": "Please tell me the status of expense number 67459."}],
  [{"role": "user", "content": "Can you upda